<a href="https://colab.research.google.com/github/Aishwarya-s-m/sumsung_project/blob/main/Research_paper_summazier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sumy
!pip install PyPDF2 python-docx openpyxl pandas

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.7 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=ef3f10ab21a007c7b11b4768e21fa056d4f4ecc25e0079da4e008249461a1504
  Stored in directory: /root/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=98c75983a22d04a4d0eec6cf67c581dfdc0f196c34caae0505f069632260b421
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built breadability docopt


In [10]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # sometimes needed in newer versions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
import re
import pandas as pd
import PyPDF2
import docx
import os
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def extract_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                # Join broken words split by newlines, e.g. "environm\nent" → "environment"
                page_text = re.sub(r'(\w+)-\s*\n(\w+)', r'\1\2', page_text)
                page_text = re.sub(r'\n', ' ', page_text)
                text += page_text + " "
    return text

def extract_from_docx(file_path):
    doc = docx.Document(file_path)
    text = " ".join([para.text for para in doc.paragraphs])
    return text

def extract_from_excel(file_path):
    df = pd.read_excel(file_path)
    text = " ".join(df.astype(str).fillna("").values.flatten())
    return text

def extract_from_text(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    return text

def summarize_file(file_path, sentence_count=60):
    """Extracts text from any file and summarizes it into given number of sentences"""
    ext = os.path.splitext(file_path)[1].lower()
    raw_text = ""

    if ext == ".pdf":
        raw_text = extract_from_pdf(file_path)
    elif ext == ".docx":
        raw_text = extract_from_docx(file_path)
    elif ext in [".xls", ".xlsx"]:
        raw_text = extract_from_excel(file_path)
    elif ext == ".txt":
        raw_text = extract_from_text(file_path)
    else:
        raise ValueError("Unsupported file format!")

    parser = PlaintextParser.from_string(raw_text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)

    # Fix spaces like "environm ent" → "environment"
    cleaned_summary = " ".join(str(sentence) for sentence in summary)
    cleaned_summary = re.sub(r'\s+', ' ', cleaned_summary)

    return cleaned_summary

In [12]:
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
print("Uploaded file:", file_path)

Saving A-Perspective-on-Environmental-Sustainability.pdf to A-Perspective-on-Environmental-Sustainability (2).pdf
Uploaded file: A-Perspective-on-Environmental-Sustainability (2).pdf


In [15]:
# Summarize into ~60 sentences (should give 2–3 pages when saved)
summary = summarize_file(file_path, sentence_count=60)

# ✅ Show only first ~10 lines in output
preview_lines = "\n".join(summary.split(". ")[:10])  # split into sentences
print("📑 Preview (first 10 lines):\n")
print(preview_lines)

# Save full 2–3 page summary
with open("summary_output.txt", "w", encoding="utf-8") as f:
    f.write(summary)

from google.colab import files
files.download("summary_output.txt")

📑 Preview (first 10 lines):

So increasingly environmental ma nagers adopt strategies aimed to prevent damage being done in the first place
And investing savings in ethical investments can help accelerate the creation of an environmentally-sustainable economy
The Commissioner acts as an independent voice that advocates, audits and reports on environmental sustainability
Natural environmentsare those where the influence of wild species (indigenous and naturalised) is dominant or very strong
social and economic) - in fact the content of ea ch domain overlaps other domains massively
1 (Link to) Commissioner for Environmental Sustainability Act 2003 2 The physical environment includes th e natural and biological environments
What exactly are we trying to maintain in the physical environment and who decides? nutrient cycl ing, the water cycle, natural water purification, climate moderation, soil protection • high quality urban environments • areas of natural beauty • other species and ecolo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>